# Introduction to Machine Learning: Task 1b

__Author__: Jannick Sicher

### Initial Configurations and Packages

In [19]:
# Load packages
import numpy as np
import matplotlib.pyplot as plot
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [20]:
# Load data
df = pd.read_csv("train.csv")
df.head()

,Id,y,x1,x2,x3,x4,x5
0,0,-5.522114,1.764052,0.400157,0.978738,2.240893,1.867558
1,1,-21.789980,-0.977278,0.950088,-0.151357,-0.103219,0.410599
2,2,-7.911497,0.144044,1.454274,0.761038,0.121675,0.443863
3,3,-3.698062,0.333674,1.494079,-0.205158,0.313068,-0.854096
4,4,-16.001833,-2.552990,0.653619,0.864436,-0.742165,2.269755


### Define Target and Predictor Variables

In [34]:
# Define Target Variable
y = df["y"] # define the target variable (dependent variable) as y

# Define Predictor Variables
x1 = df["x1"]
x2 = df["x2"]
x3 = df["x3"]
x4 = df["x4"]
x5 = df["x5"]

# Define all Variable Transformations
Xs = np.column_stack((x1, x2, x3, x4, x5, x1**2, x2**2, x3**2, x4**2, x5**2,
                       np.exp(x1),np.exp(x2),np.exp(x3),np.exp(x4),np.exp(x5), 
                        np.cos(x1),np.cos(x2),np.cos(x3),np.cos(x4),np.cos(x5),
                        np.ones(df.shape[0])))

###  Regression Coefficients

In [35]:
# Define linear_model
linear_model = LinearRegression()

# Running the regression for defined variables
linear_model.fit(Xs,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [36]:
# Access all coefficients and store them in dataframe
coefficients = model.coef_
submission = pd.DataFrame(data = coefficients, columns=None)
submission

,0
0,0.621003
1,-1.880502
2,0.384095
3,-0.441386
4,0.386888
5,-0.361083
6,0.391405
7,0.175731
8,-2.719074
9,2.234341
